In [1]:
!pip install -U efficientnet

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install numpy==1.21

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from keras import applications
from keras import callbacks
from keras.models import Sequential

2023-07-26 09:45:51.742761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 09:45:52.221391: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 09:45:52.502028: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import efficientnet.keras as efn

model = efn.EfficientNetB7(weights='imagenet')

2023-07-26 09:45:58.478508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 09:46:00.808088: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30976 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:af:00.0, compute capability: 7.0


In [6]:
!pip install tqdm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers

In [8]:
train_dir = "Emergency_Vehicles/train"
test_dir = "Emergency_Vehicles/test"
train_df = pd.read_csv('Emergency_Vehicles/train.csv')
train_df.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [9]:
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout

from keras.callbacks import EarlyStopping




In [10]:
import time
def upload_model():
    eff_net = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    
    #datagen=ImageDataGenerator(rescale=1./255)
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,      # range (0-180) within which to randomly rotate pictures
        width_shift_range=0.2,  # fraction of total width to randomly translate pictures
        height_shift_range=0.2, # fraction of total height to randomly translate pictures
        shear_range=0.2,        # randomly applying shear transformations
        zoom_range=0.2,         # randomly zooming inside pictures
        horizontal_flip=True,   # randomly flipping half of the images horizontally
        fill_mode='nearest'     # strategy used for filling in newly created pixels
    )

    batch_size=150
    
    train_df.emergency_or_not=train_df.emergency_or_not.astype(str)
    
    train_generator=datagen.flow_from_dataframe(dataframe=train_df[:1150],directory=train_dir,x_col='image_names',
                                            y_col='emergency_or_not',class_mode='binary',batch_size=batch_size,
                                            target_size=(64,64))


    validation_generator=datagen.flow_from_dataframe(dataframe=train_df[1151:],directory=train_dir,x_col='image_names',
                                                    y_col='emergency_or_not',class_mode='binary',batch_size=50,
                                                    target_size=(64,64))
    
    efficient_net = efn.EfficientNetB7(
    weights='imagenet',
    input_shape=(64,64,3),
    include_top=False,
    pooling='max'
)

    model = Sequential()
    model.add(efficient_net)
    model.add(Dense(units = 120, activation='relu'))
    model.add(Dropout(0.5))  # randomly sets 50% of input units to 0 at each update during training time
    model.add(Dense(units = 120, activation = 'relu'))
    model.add(Dropout(0.5))  # randomly sets 50% of input units to 0 at each update during training time
    model.add(Dense(units = 1, activation='sigmoid'))
    model.summary()
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    start = time.time()

    early_stop = EarlyStopping(monitor='val_loss', patience=5)  # stop training after the validation loss stops improving for 5 epochs
    history = model.fit(
        train_generator,
        epochs=50,
        steps_per_epoch=8,
        validation_data=validation_generator,
        validation_steps=7,
        callbacks=[early_stop]  # early stopping
    )

    end = time.time()

    print(f"Training time: {end - start} seconds")
    
    return model

In [11]:
model = upload_model()

Found 1150 validated image filenames belonging to 2 classes.
Found 495 validated image filenames belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b7 (Functional  (None, 2560)             64097680  
 )                                                               
                                                                 
 dense (Dense)               (None, 120)               307320    
                                                                 
 dropout (Dropout)           (None, 120)               0         
                                                                 
 dense_1 (Dense)             (None, 120)               14520     
                                                                 
 dropout_1 (Dropout)         (None, 120)               0         
                                                                 
 

2023-07-26 09:47:04.257650: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


8/8 [==============================] - 54s 2s/step - loss: 0.7721 - accuracy: 0.5122 - val_loss: 0.7341 - val_accuracy: 0.3686
Epoch 2/50
8/8 [==============================] - 4s 488ms/step - loss: 0.7482 - accuracy: 0.5148 - val_loss: 0.7344 - val_accuracy: 0.3886
Epoch 3/50
8/8 [==============================] - 4s 473ms/step - loss: 0.7306 - accuracy: 0.5287 - val_loss: 0.7336 - val_accuracy: 0.3714
Epoch 4/50
8/8 [==============================] - 4s 525ms/step - loss: 0.7137 - accuracy: 0.5591 - val_loss: 0.7169 - val_accuracy: 0.4400
Epoch 5/50
8/8 [==============================] - 4s 524ms/step - loss: 0.6822 - accuracy: 0.5843 - val_loss: 0.7130 - val_accuracy: 0.4229
Epoch 6/50
8/8 [==============================] - 4s 485ms/step - loss: 0.7140 - accuracy: 0.5548 - val_loss: 0.6984 - val_accuracy: 0.4657
Epoch 7/50
8/8 [==============================] - 4s 493ms/step - loss: 0.7039 - accuracy: 0.5635 - val_loss: 0.6921 - val_accuracy: 0.5314
Epoch 8/50
8/8 [=================

In [24]:
from tensorflow.keras.preprocessing import image

def classify_image(image_path):
        img = image.load_img(image_path, target_size=(64, 64))
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        x = x / 255.0

        pred = model.predict(x)

        return 1 if pred>0.75 else 0


In [26]:
print(classify_image("testCar1.png"))
print(classify_image("testCar2.png"))

1/1 [==============================] - 0s 48ms/step
1
1/1 [==============================] - 0s 43ms/step
0
